#### 네이버 쇼핑 리뷰 크롤링

In [ ]:
# pip install selenium==4.2.0
# pip install numpy==1.23.0
# pip install pandas==2.0.3
# pip install konlpy==0.6.0
# pip install matplotlib==3.6.0
# pip install seaborn==0.12.1
# pip install wordcloud==1.9.2

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

import requests
from bs4 import BeautifulSoup
 
import pandas as pd
from datetime import datetime
import time
 
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [10]:
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

In [11]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(3)

In [12]:
name=['멜라스틱']
category=['리뷰/별점']

#후기 링크
ns_address="https://search.shopping.naver.com/catalog/27502836523?&NaPm=ct%3Dlwsmel9s%7Cci%3D8683475f17b7c100c5b04ecff0eb09d44db599ae%7Ctr%3Dslcc%7Csn%3D95694%7Chk%3Dad91c5d90d1f00a11de0dda1c0a946087ca92aab"
#xpath
shoppingmall_review= "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul"

In [15]:
header = {'User-Agent': ''}
service = Service(ChromeDriverManager().install())
d = webdriver.Chrome(service=service)
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

element=d.find_element(By.XPATH, shoppingmall_review)
d.execute_script("arguments[0].click();", element)
sleep(2)

In [14]:
# 리뷰 가져오기
d.find_element(By.XPATH, shoppingmall_review).click() #스크롤 건드리면 안됨
name_=name[0]
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1

In [16]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from time import sleep

# 초기화
page = 1
cnt = 1
reviews = []
stars = []

while page <= 94:
    j = 1
    print("페이지", page, "\n") 
    sleep(2)
    while True:
        try:
            star_selector = f'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li[{j}]/div[1]/span[1]'
            review_selector = f'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li[{j}]/div[2]/div[1]/p'
            
            star = d.find_element(By.XPATH, star_selector).text
            review = d.find_element(By.XPATH, review_selector).text
            
            stars.append(star)
            reviews.append(review)
            
            if j % 2 == 0:
                ELEMENT = d.find_element(By.XPATH, review_selector)
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                
            j += 1
            print(cnt, review, star, "\n")
            cnt += 1
        except NoSuchElementException:
            break  # 리뷰가 더 이상 없을 때 반복 종료
    
    sleep(2)

    if page<11:#page10
        try: #리뷰의 마지막 페이지에서 error발생
            page +=1
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page)+']').click() 
        except: break #리뷰의 마지막 페이지에서 process 종료

    else : 
        try: #page11부터
            page+=1
            if page%10==0:  # 현재 19페이지, 다음 20페이지(=page) 예정일 경우 '이후' 버튼 바로 앞에 있는 버튼을 누른다
                next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[11]').click()
            elif page%10==1:    # 현재 20페이지, 다음 21페이지(=page) 예정일 경우 '이후' 버튼을 누른다
                next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[12]').click()
            else:   # 그 외: 현재 11페이지, 다음 12페이지(=page) 예정일 경우, a[3]을 눌러야 한다.
                next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page%10+1)+']').click()
        except NoSuchElementException:
            break  # 다음 페이지가 없을 때 반복 종료

df_멜라스틱= add_dataframe(name_, category_, reviews, stars, cnt)
# save()


페이지 1 

1 일단 립밤도 불안해요 디자인이 여성뷴들이 좋아할만하지만 스틱끝부분이 너무지져분해지고 깔끔하지못해요 아마도 사각이라서 그럴수도있고 사은품으로주신립밥은 샘플인가요 본품인가요 역시 불안한디자인이에요 작고귀여운데 스틱이 똑부러질거같은 아찔한 하이힐을보는거같아 좀 불안했어요 디자인 컬러나 그립감은좋았으나 바르고나서 화장품이 약간 뭉개져있으면 다음에도 또 쓰고싶은생각이드나 생각합니다 커버 밀찰력 우수합니다 눈에보이는 디자인말고 그립갑이나 화장품 넣는 용기를 좀더 생각하시면 대박날듯해요 아참 그리고 부러쉬부분이 뚜껑에 끼어서 삐져나와서 약간 실망했어요 평점4 

2 일단 커버력은 보통이라고 후기를.보고 샀어용 너무 과한거는 싫구 가끔 뾰루지도 나는 피부 입니다 근데 마스크를 오래쓰다보니 진한건 싫더라구요~ 톤업크림처럼 약간의 보정되구요 건조한편이긴해요 스틱이니까요~ 기초 를 탄탄히 하면 건조하진 않을거같구요 묻어남도 적네요 제가원하던거 입니다
첫번째 사진은>>>스틱부분이 추워서 그런지 약간 갈라져 있어요. 쓰는데는 지장 없구요
두번째 사진>>> 손등에 쓱쓱 문질러 봤어요 핑크빛 돌아요
세번째 사진>>>문지르니 자기 피부색처럼 변해요 생기 있어보이는거같아요 마스크를 쓴다해도 너무 쌩얼은 부담스러울때 좋을것 같아요! 평점5 

3 방송보고 어떨까 하는 마음에 구매 했는데
너무 좋아요
쿠션 정말 좋다라는것 다 써봤는데 커버력은 쏘쏘하지만
제일 좋은점이 얼굴위에 바른 느낌이 덜 하다는게
자주자주 사용 할것 같아요 요즘들어 제일 맘에 드네요
다사용하고 더 많은 패키지로 구매 해야 할것 같아요 평점4 

4 커버력 불만이라는 후기 보고 망설였는데, 커버력 좋은데요~ 피부도 좋아보이고 촉촉해서 좋아요.
폭염이라 땀 많이나면 뽀송한 느낌은 덜 하지만 그만큼 광이 나요. 하얀색 스틱인데 피부에 바르면 자연스럽게 어우러지고 바르기도 간편하고 좋습니다. 브러쉬로 바를 때도 있고 급하면 손가락으로 바르는데 둘다 괜찮아요.^^ 다 쓰면 또 구매할래요. 평점5 

5 아직사용은 

In [17]:
df_멜라스틱

,type,category,review,star
1,멜라스틱,리뷰/별점,일단 립밤도 불안해요 디자인이 여성뷴들이 좋아할만하지만 스틱끝부분이 너무지져분해지고...,평점4
2,멜라스틱,리뷰/별점,일단 커버력은 보통이라고 후기를.보고 샀어용 너무 과한거는 싫구 가끔 뾰루지도 나는...,평점5
3,멜라스틱,리뷰/별점,방송보고 어떨까 하는 마음에 구매 했는데\n너무 좋아요\n쿠션 정말 좋다라는것 다 ...,평점4
4,멜라스틱,리뷰/별점,"커버력 불만이라는 후기 보고 망설였는데, 커버력 좋은데요~ 피부도 좋아보이고 촉촉해...",평점5
5,멜라스틱,리뷰/별점,아직사용은 안했지만 만족스럽지는 않네요\n검수 안하나요?\n스틱이 밖으로 돌출되어 ...,평점4
...,...,...,...,...
1871,멜라스틱,리뷰/별점,좋아용 좋아용 좋아용 좋아용 좋아용,평점5
1872,멜라스틱,리뷰/별점,조아요 조아요 조아요 조아요 조아요 조아요,평점5
1873,멜라스틱,리뷰/별점,만 족 팝 니 다 .,평점5
1874,멜라스틱,리뷰/별점,.....,평점4


In [18]:
df_멜라스틱.to_csv("크롤링데이터_멜라스틱.csv", encoding = 'utf-8-sig', header= False)

----------------------------------------------------------------------------------------------------------------------

#### 데이터 전처리

In [19]:
import pandas as pd

df_멜라스틱 = pd.read_csv('크롤링데이터_멜라스틱.csv', names=['item', 'category', 'review', 'rating'])

In [20]:
print(df_멜라스틱.shape)
df_멜라스틱.head()

(1875, 4)


,item,category,review,rating
1,멜라스틱,리뷰/별점,일단 립밤도 불안해요 디자인이 여성뷴들이 좋아할만하지만 스틱끝부분이 너무지져분해지고...,평점4
2,멜라스틱,리뷰/별점,일단 커버력은 보통이라고 후기를.보고 샀어용 너무 과한거는 싫구 가끔 뾰루지도 나는...,평점5
3,멜라스틱,리뷰/별점,방송보고 어떨까 하는 마음에 구매 했는데\n너무 좋아요\n쿠션 정말 좋다라는것 다 ...,평점4
4,멜라스틱,리뷰/별점,"커버력 불만이라는 후기 보고 망설였는데, 커버력 좋은데요~ 피부도 좋아보이고 촉촉해...",평점5
5,멜라스틱,리뷰/별점,아직사용은 안했지만 만족스럽지는 않네요\n검수 안하나요?\n스틱이 밖으로 돌출되어 ...,평점4


In [21]:
# 중복된 행 제거
df_멜라스틱.drop_duplicates('review',keep='first')
df_멜라스틱.shape

(1875, 4)

In [22]:
#불필요한 행 제거
df_멜라스틱 = df_멜라스틱[['review','rating']].copy()
df_멜라스틱

,review,rating
1,일단 립밤도 불안해요 디자인이 여성뷴들이 좋아할만하지만 스틱끝부분이 너무지져분해지고...,평점4
2,일단 커버력은 보통이라고 후기를.보고 샀어용 너무 과한거는 싫구 가끔 뾰루지도 나는...,평점5
3,방송보고 어떨까 하는 마음에 구매 했는데\n너무 좋아요\n쿠션 정말 좋다라는것 다 ...,평점4
4,"커버력 불만이라는 후기 보고 망설였는데, 커버력 좋은데요~ 피부도 좋아보이고 촉촉해...",평점5
5,아직사용은 안했지만 만족스럽지는 않네요\n검수 안하나요?\n스틱이 밖으로 돌출되어 ...,평점4
...,...,...
1871,좋아용 좋아용 좋아용 좋아용 좋아용,평점5
1872,조아요 조아요 조아요 조아요 조아요 조아요,평점5
1873,만 족 팝 니 다 .,평점5
1874,.....,평점4


In [23]:
# 'rating' 컬럼의 값을 숫자로 변환
df_멜라스틱['rating'] = df_멜라스틱['rating'].apply(lambda x: int(re.findall(r'\d+', x)[0]) if re.findall(r'\d+', x) else None)

df_멜라스틱

,review,rating
1,일단 립밤도 불안해요 디자인이 여성뷴들이 좋아할만하지만 스틱끝부분이 너무지져분해지고...,4
2,일단 커버력은 보통이라고 후기를.보고 샀어용 너무 과한거는 싫구 가끔 뾰루지도 나는...,5
3,방송보고 어떨까 하는 마음에 구매 했는데\n너무 좋아요\n쿠션 정말 좋다라는것 다 ...,4
4,"커버력 불만이라는 후기 보고 망설였는데, 커버력 좋은데요~ 피부도 좋아보이고 촉촉해...",5
5,아직사용은 안했지만 만족스럽지는 않네요\n검수 안하나요?\n스틱이 밖으로 돌출되어 ...,4
...,...,...
1871,좋아용 좋아용 좋아용 좋아용 좋아용,5
1872,조아요 조아요 조아요 조아요 조아요 조아요,5
1873,만 족 팝 니 다 .,5
1874,.....,4


In [24]:
#정규표현식 사용하여 한글 제외 모두 제거

df_멜라스틱['review'] = df_멜라스틱['review'].replace(r'^\s*$', np.nan, regex=True)

In [25]:
# 한글제외 후 nan값만 남은 행 제거

df_멜라스틱 = df_멜라스틱.dropna(subset=['review'])
df_멜라스틱

,review,rating
1,일단 립밤도 불안해요 디자인이 여성뷴들이 좋아할만하지만 스틱끝부분이 너무지져분해지고...,4
2,일단 커버력은 보통이라고 후기를.보고 샀어용 너무 과한거는 싫구 가끔 뾰루지도 나는...,5
3,방송보고 어떨까 하는 마음에 구매 했는데\n너무 좋아요\n쿠션 정말 좋다라는것 다 ...,4
4,"커버력 불만이라는 후기 보고 망설였는데, 커버력 좋은데요~ 피부도 좋아보이고 촉촉해...",5
5,아직사용은 안했지만 만족스럽지는 않네요\n검수 안하나요?\n스틱이 밖으로 돌출되어 ...,4
...,...,...
1871,좋아용 좋아용 좋아용 좋아용 좋아용,5
1872,조아요 조아요 조아요 조아요 조아요 조아요,5
1873,만 족 팝 니 다 .,5
1874,.....,4


In [26]:
df_멜라스틱.to_csv("크롤링데이터.csv", encoding = 'utf-8-sig')